In [1]:
import os
import re
import json
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup


In [22]:
df = pd.read_csv("Files/apartmentsPortugal.csv")
df

,URL,Titles,Price,Location,Rooms,Areas,Bathrooms,Listing Type,Useful area,Gross area,Construction year,Energetic certificate,Enterprise
0,https://www.imovirtual.com/pt/anuncio/apartame...,Apartamento T2+1 c/ Lugar de Garagem e Arrumo ...,260 000 €,"Mafamude e Vilar do Paraíso, Vila Nova de Gaia...",T3,106 m²,2,Usado,106 m²,121 m²,2001.0,C,NaN
1,https://www.imovirtual.com/pt/anuncio/apartame...,Apartamento T1 para venda na Praia da Rocha,185 000 €,"Portimão, Faro",T1,"58,30 m²",1,Usado,"58,30 m²","68,30 m²",2001.0,D,não
2,https://www.imovirtual.com/pt/anuncio/apartame...,"Apartamento T4, 2 Suítes, Luxo, Fonte Nova, Av...",950 000 €,"Glória e Vera Cruz, Aveiro",T4,175 m²,4 ou mais,Em construção,175 m²,245 m²,2023.0,A+,sim
3,https://www.imovirtual.com/pt/anuncio/apartame...,Apartamento T4,780 000 €,"Vila do Conde, Porto",T4,"181,65 m²",NaN,Em construção,"181,65 m²","207,90 m²",NaN,B,sim
4,https://www.imovirtual.com/pt/anuncio/venda-de...,Venda de Apartamento T3 duplex no centro de Vi...,300 000 €,"Vila do Conde, Porto",T3,183 m²,2,NaN,183 m²,NaN,NaN,D,não
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,https://www.imovirtual.com/pt/anuncio/t1-novo-...,T1 Novo ao ISCAP,175 000 €,"São Mamede de Infesta e Senhora da Hora, Matos...",T1,55 m²,NaN,Novo,55 m²,NaN,NaN,A,não
89996,https://www.imovirtual.com/pt/anuncio/apartame...,"Apartamento T2 Venda em Aldoar, Foz do Douro e...",625 000 €,"Aldoar, Foz do Douro e Nevogilde, Porto",T2,"78,80 m²",2,Usado,"78,80 m²","78,80 m²",NaN,Isento / Em Trâmite,não
89997,https://www.imovirtual.com/pt/anuncio/t3-1-em-...,T3+1 em construção com excelentes acabamentos ...,470 000 €,"Montijo e Afonsoeiro, Montijo, Setúbal",T4,140 m²,3,Em construção,140 m²,190 m²,NaN,A+,não
89998,https://www.imovirtual.com/pt/anuncio/apartame...,APARTAMENTO T3 NOVO NO CENTRO DE LEIRIA,290 000 €,"Leiria, Pousos, Barreira e Cortes, Leiria",T3,"138,70 m²",2,Em construção,"138,70 m²",NaN,NaN,Isento / Em Trâmite,não


In [ ]:
base_url = "https://www.imovirtual.com/comprar/apartamento/?page="
num_pages = 3700  # Specify the number of pages to scrape
df = []
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

In [ ]:
def scrape_data(url):
    response = requests.get(url, headers=headers)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    article_elements = soup.find_all('article')

    titles = []
    prices = []
    locations = []
    rooms = []
    areas = []
    bathrooms = []
    listing_types = []
    urls = []


    for article_element in article_elements:
        # Extract title
        title_element = article_element.find('span', class_='offer-item-title')
        title = title_element.text.strip() if title_element else np.nan
        titles.append(title)

        # Extract URL
        url_element = article_element.find('a')
        url = url_element['href'] if url_element else np.nan
        urls.append(url)

        # Extract price
        price_element = article_element.find(class_='offer-item-price')
        price = price_element.text.strip() if price_element else np.nan
        prices.append(price)

        # Extract location
        location_element = article_element.find('p', class_='text-nowrap')
        location = location_element.text.strip().split(':')[-1].strip() if location_element else np.nan
        locations.append(location)

        # Extract room
        room_element = article_element.find(class_='offer-item-rooms')
        room = room_element.text.strip() if room_element else np.nan
        rooms.append(room)

        # Extract area
        area_element = article_element.find(class_='offer-item-area')
        area = area_element.text.strip() if area_element else np.nan
        areas.append(area)

        # Extract bathrooms and listing type
        details_element = article_element.find(class_=['parameters-view', 'params-small'])
        bathroom = np.nan
        listing_type = np.nan

        if details_element:
            li_elements = details_element.find_all('li')
            for li_element in li_elements:
                text = li_element.text.strip()
                if 'Casas de Banho' in text:
                    bathroom = text.split(':')[-1].strip()
                elif text in ['Em construção', 'Usado', 'Novo', 'Remodelado', 'Ruína', 'Para recuperar']:
                    listing_type = text

        bathrooms.append(bathroom)
        listing_types.append(listing_type)


    return titles, prices, locations, rooms, areas, bathrooms, listing_types, urls


In [ ]:

titles = []
prices = []
locations = []
rooms = []
areas = []
bathrooms = []
listing_types = []
urls = []

for page in range(1, num_pages + 1):
    print(page)
    page_url = base_url + str(page)
    (
        page_titles,
        page_prices,
        page_locations,
        page_rooms,
        page_areas,
        page_bathrooms,
        page_listing_types,
        page_urls,
    ) = scrape_data(page_url)

    titles += page_titles
    prices += page_prices
    locations += page_locations
    rooms += page_rooms
    areas += page_areas
    bathrooms += page_bathrooms
    listing_types += page_listing_types
    urls += page_urls

    print(
        f"Page {page}: Titles: {len(page_titles)}, Prices: {len(page_prices)}, ..."
    )

In [ ]:
len(listing_types)

In [ ]:
# Create a DataFrame
data = {
    "URL": urls,
    "Titles": titles,
    "Price": prices,
    "Location": locations,
    "Rooms": rooms,
    "Areas": areas,
    "Bathrooms": bathrooms,
    "Listing Type": listing_types,
}

df = pd.DataFrame(data)
# Print the DataFrame
df

In [ ]:
df.to_csv("Files/apartmentsPortugal.csv", index="0")

In [ ]:
from fake_useragent import UserAgent
import time
import random

# Load your DataFrame containing property URLs
df = pd.read_csv('Files/apartmentsPortugal.csv')

# Create empty columns in the existing DataFrame to store the extracted information
df['Useful area'] = None
df['Gross area'] = None
df['Construction year'] = None
df['Energetic certificate'] = None
df['Enterprise'] = None
df['Description'] = None

# Initialize UserAgent object
ua = UserAgent()

url_counter = 0

for property_number, url in enumerate(df['URL']):
    # Send a request to the URL with a random User-Agent
    headers = {'User-Agent': ua.random}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        try:
            # Extract information from the HTML
            useful_area = soup.find('div', {'aria-label': 'Área útil (m²)'}).find('div', {'class': 'css-1ytkscc'}).text.strip()
            df.at[property_number, 'useful_area'] = useful_area
        except AttributeError:
            pass

        try:
            gross_area = soup.find('div', {'aria-label': 'Área bruta (m²)'}).find('div', {'class': 'css-1ytkscc'}).text.strip()
            df.at[property_number, 'gross_area'] = gross_area
        except AttributeError:
            pass

        try:
            construction_year = soup.find('div', {'aria-label': 'Ano de construção'}).find('div', {'class': 'css-1ytkscc'}).text.strip()
            df.at[property_number, 'construction_year'] = construction_year
        except AttributeError:
            pass

        try:
            energetic_certificate = soup.find('div', {'aria-label': 'Certificado Energético'}).find('div', {'class': 'css-1ytkscc'}).text.strip()
            df.at[property_number, 'energetic_certificate'] = energetic_certificate
        except AttributeError:
            pass

        try:
            enterprise = soup.find('div', {'aria-label': 'Empreendimento'}).find('div', {'class': 'css-1ytkscc'}).text.strip()
            df.at[property_number, 'enterprise'] = enterprise
        except AttributeError:
            pass

        try:
            description = soup.find('div', {'data-cy': 'adPageAdDescription'}).text.strip()
            df.at[property_number, 'description'] = description
        except AttributeError:
            pass

        # Introduce a random delay between requests (e.g., between 0.5 and 1.5 seconds)
        delay = random.uniform(0.5, 1.5)
        time.sleep(delay)
        # Increment the URL counter
        url_counter += 1

        # Check if 100 URLs have been processed and save the DataFrame
        if url_counter % 100 == 0:
            df.to_csv('Files/apartmentsPortugal.csv', index=False)
            print(f"Saved progress at {url_counter} URLs.")

    else:
        print(f"Failed to retrieve property page for {url}")

# Save the updated DataFrame to the existing CSV file
df.to_csv('Files/apartmentsPortugal.csv', index=False)

print("Information extracted and added to apartmentsPortugal.csv.")


In [ ]:
# Load your DataFrame containing property URLs
df = pd.read_csv('Files/apartmentsPortugal.csv')

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

for property_number, url in enumerate(df['URL']):
    # Send a request to the URL
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the script tag containing image data
        script_elements = soup.find_all('script')

        for script_element in script_elements:
            script_content = script_element.string
            if script_content:
                # Use regex to extract image URLs from the script
                image_urls = re.findall(r'"large":"(https://ireland\.apollo\.olxcdn\.com[^"]+)"', script_content)
                if image_urls:
                    property_folder = f'Files/PropertiesImages/Property{property_number}'
                    os.makedirs(property_folder, exist_ok=True)

                    for index, image_url in enumerate(image_urls):
                        image_response = requests.get(image_url)
                        if image_response.status_code == 200:
                            image_extension = "webp"  # Assuming images are in webp format
                            image_filename = f"Property{property_number}-Image{index + 1}.{image_extension}"
                            image_path = os.path.join(property_folder, image_filename)

                            with open(image_path, 'wb') as f:
                                f.write(image_response.content)
                            print(f"Image {index + 1} saved to {image_path}")
                        else:
                            print(f"Failed to download image {index + 1}")
                else:
                    print("No image URLs found in script")
    else:
        print(f"Failed to retrieve property page for {url}")

print("Images downloaded and saved for all properties.")


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import re

# Load your DataFrame containing property URLs
df = pd.read_csv('Files/apartmentsPortugal.csv')

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Choose  a property to start from
start_property = 13843  

for property_number, url in enumerate(df['URL']):
    if property_number < start_property:
        continue  # Pula propriedades até atingir a propriedade inicial

    # Send a request to the URL
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the script tag containing image data
        script_elements = soup.find_all('script')

        for script_element in script_elements:
            script_content = script_element.string
            if script_content:
                # Use regex to extract image URLs from the script
                image_urls = re.findall(r'"large":"(https://ireland\.apollo\.olxcdn\.com[^"]+)"', script_content)
                if image_urls:
                    property_folder = f'Files/PropertiesImages/Property{property_number}'
                    os.makedirs(property_folder, exist_ok=True)

                    for index, image_url in enumerate(image_urls):
                        image_response = requests.get(image_url)
                        if image_response.status_code == 200:
                            image_extension = "webp"  # Assuming images are in webp format
                            image_filename = f"Property{property_number}-Image{index + 1}.{image_extension}"
                            image_path = os.path.join(property_folder, image_filename)

                            with open(image_path, 'wb') as f:
                                f.write(image_response.content)
                            print(f"Image {index + 1} saved to {image_path}")
                        else:
                            print(f"Failed to download image {index + 1}")
                else:
                    print("No image URLs found in script")
    else:
        print(f"Failed to retrieve property page for {url}")

print("Images downloaded and saved for all properties.")
